In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm_notebook 

import skimage.io, skimage.color
import matplotlib.pyplot
import HOG
from skimage.feature import greycomatrix, greycoprops
from sklearn.preprocessing import StandardScaler

In [ ]:
def calculate_gradient(img, template):
    ts = template.size #Number of elements in the template (3).
    #New padded array to hold the resultant gradient image.
    new_img = np.zeros((img.shape[0]+ts-1, 
                           img.shape[1]+ts-1))
    new_img[np.uint16((ts-1)/2.0):img.shape[0]+np.uint16((ts-1)/2.0), 
            np.uint16((ts-1)/2.0):img.shape[1]+np.uint16((ts-1)/2.0)] = img
    result = np.zeros((new_img.shape))
    
    for r in np.uint16(np.arange((ts-1)/2.0, img.shape[0]+(ts-1)/2.0)):
        for c in np.uint16(np.arange((ts-1)/2.0, 
                              img.shape[1]+(ts-1)/2.0)):
            curr_region = new_img[r-np.uint16((ts-1)/2.0):r+np.uint16((ts-1)/2.0)+1, 
                                  c-np.uint16((ts-1)/2.0):c+np.uint16((ts-1)/2.0)+1]
            curr_result = curr_region * template
            score = np.sum(curr_result)
            result[r, c] = score
    #Result of the same size as the original image after removing the padding.
    result_img = result[np.uint16((ts-1)/2.0):result.shape[0]-np.uint16((ts-1)/2.0), 
                        np.uint16((ts-1)/2.0):result.shape[1]-np.uint16((ts-1)/2.0)]
    return result_img

def gradient_magnitude(horizontal_gradient, vertical_gradient):
    horizontal_gradient_square = np.power(horizontal_gradient, 2)
    vertical_gradient_square = np.power(vertical_gradient, 2)
    sum_squares = horizontal_gradient_square + vertical_gradient_square
    grad_magnitude = np.sqrt(sum_squares)
    return grad_magnitude

def gradient_direction(horizontal_gradient, vertical_gradient):
    grad_direction = np.arctan(vertical_gradient/(horizontal_gradient+0.00000001))
    grad_direction = np.rad2deg(grad_direction)
    grad_direction = grad_direction%180
    return grad_direction

def HOG_cell_histogram(cell_direction, cell_magnitude, hist_bins):
    HOG_cell_hist = np.zeros(shape=(hist_bins.size))
    cell_size = cell_direction.shape[0]
    
    for row_idx in range(cell_size):
        for col_idx in range(cell_size):
            curr_direction = cell_direction[row_idx, col_idx]
            curr_magnitude = cell_magnitude[row_idx, col_idx]
    
            diff = np.abs(curr_direction - hist_bins)
            
            if curr_direction < hist_bins[0]:
                first_bin_idx = 0
                second_bin_idx = hist_bins.size-1
            elif curr_direction > hist_bins[-1]:
                first_bin_idx = hist_bins.size-1
                second_bin_idx = 0
            else:
                first_bin_idx = np.where(diff == np.min(diff))[0][0]
                temp = hist_bins[[(first_bin_idx-1)%hist_bins.size, (first_bin_idx+1)%hist_bins.size]]
                temp2 = np.abs(curr_direction - temp)
                res = np.where(temp2 == np.min(temp2))[0][0]
                if res == 0 and first_bin_idx != 0:
                    second_bin_idx = first_bin_idx-1
                else:
                    second_bin_idx = first_bin_idx+1
            
            first_bin_value = hist_bins[first_bin_idx]
            second_bin_value = hist_bins[second_bin_idx]
            HOG_cell_hist[first_bin_idx] = HOG_cell_hist[first_bin_idx] + (np.abs(curr_direction - first_bin_value)/(180.0/hist_bins.size)) * curr_magnitude
            HOG_cell_hist[second_bin_idx] = HOG_cell_hist[second_bin_idx] + (np.abs(curr_direction - second_bin_value)/(180.0/hist_bins.size)) * curr_magnitude
    return HOG_cell_hist


In [ ]:
path = r"C:\Users\Dell\Desktop\Project2\Data"
X = []

for file in tqdm_notebook(os.listdir(path)): 

	feat_set = []
	im = skimage.io.imread(path+'\\'+file)
	img = skimage.color.rgb2gray(im)

	horizontal_mask = np.array([-1, 0, 1])
	vertical_mask = np.array([[-1],[0],[1]])
	                             

	horizontal_gradient = HOG.calculate_gradient(img, horizontal_mask)
	vertical_gradient = HOG.calculate_gradient(img, vertical_mask)

	grad_magnitude = HOG.gradient_magnitude(horizontal_gradient, vertical_gradient)
	grad_direction = HOG.gradient_direction(horizontal_gradient, vertical_gradient)

	grad_direction = grad_direction % 180
	hist_bins = np.array([10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170])

	# Histogram of the first cell in the first block.
	cell_direction = grad_direction[:8, :8]
	cell_magnitude = grad_magnitude[:8, :8]
	HOG_cell_hist = HOG.HOG_cell_histogram(cell_direction, cell_magnitude, hist_bins)
	
	# GLCM Feature Generation
	im_glcm = Image.open(path+'\\'+file)
	im_glcm = im_glcm.convert("L")
	im_glcm = np.array(im_glcm.copy())
	g = greycomatrix(im_glcm,[1],[0],symmetric=False,normed=True)
	properties = ['contrast','energy','homogeneity','correlation','dissimilarity']
	glcm_features = [greycoprops(g,prop)[0][0] for prop in properties]
	glcm_features = np.array(glcm_features)

	feat_set = np.concatenate((HOG_cell_hist,glcm_features))

	X.append(feat_set)

X = np.array(X)
print(X)
print(X.shape)


In [ ]:
Y1 = np.zeros(933,dtype=int)
Y2 = np.ones(912,dtype=int)
Y = np.concatenate([Y1,Y2])
print(Y)

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
print(X)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,stratify=Y,random_state=1,test_size=0.1)

In [ ]:
class Perceptron:
    
    def __init__(self):
        self.w = None
        self.b = None
    def model(self,x):
        return (np.dot(self.w,x) >= self.b)
    def predict(self,X):
        Y = []
        for x in X:
            Y.append(self.model(x))
        return np.array(Y)
    def fit(self,X,Y,epochs=1,lr=1):
        
        self.w = np.ones(X.shape[1])
        self.b = 0
        wt_matrix = []
        for e in tqdm_notebook(range(epochs),total=epochs,unit="epoch"):
            accuracy = {}
            max_accuracy = 0
            for x,y in zip(X,Y):
                y_pred = self.model(x)
                if y_pred == 0 and y == 1:
                    self.w = self.w + lr*x
                    self.b = self.b + lr*1
                elif y_pred == 1 and y == 0:
                    self.w = self.w - lr*x
                    self.b = self.b - lr*1
            accuracy[e] = accuracy_score(self.predict(X),Y)
            if accuracy[e] > max_accuracy:
                max_accuracy = accuracy[e]
                checkpw = self.w
                checkpb = self.b
        print("Maximum accuracy achived on training data is",max_accuracy)
        

In [ ]:
perceptron = Perceptron()
perceptron.fit(X,Y,10000,0.01)

In [ ]:
perceptron.w

In [ ]:
Y_pred_test = perceptron.predict(X_test)
print("Test accuracy ",accuracy_score(Y_pred_test,Y_test))